In [15]:
import torch
from PIL import Image
import sys
import os

import warnings
warnings.filterwarnings("ignore")


project_root = os.getcwd()
src_path = os.path.join(project_root, '../src')
sys.path.insert(0, project_root)
sys.path.insert(0, src_path)

import open_clip

# Call PanDerm2 with huggingface checkpoint
model, _, preprocess = open_clip.create_model_and_transforms('hf-hub:redlessone/PanDerm2')
model = model.to('cpu')
model.eval()

# Similarly, we init the tokenizer with huggingface checkpoint
tokenizer = open_clip.get_tokenizer('hf-hub:redlessone/PanDerm2')

# Read example image
image = preprocess(Image.open("PAT_8_15_820.png")).unsqueeze(0)

# Here we will use the disease label in PAD dataset to build text prompts
PAD_CLASSNAMES= ["nevus",  "basal cell carcinoma","actinic keratosis", "seborrheic keratosis","squamous cell carcinoma","melanoma"]
template = lambda c: f'This is a skin image of {c}'

text = tokenizer([template(c) for c in PAD_CLASSNAMES])

with torch.no_grad(), torch.autocast("cpu"):
    image_features = model.encode_image(image) # Encode image to visual feature
    text_features = model.encode_text(text) # Encode text to text feature
    
    # Normalize
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Compute similarity
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

final_prediction = PAD_CLASSNAMES[torch.argmax(text_probs[0])]

print(f'This images is diagnosised as {final_prediction}.')

print("\nLabel probs: ")
for idx, label in enumerate(PAD_CLASSNAMES):
    print(f"{PAD_CLASSNAMES[idx]}: {text_probs[:, idx].item():.3f}")

Using default Setting For PanDerm-Large
This images is diagnosised as basal cell carcinoma.

Label probs: 
nevus: 0.000
basal cell carcinoma: 1.000
actinic keratosis: 0.000
seborrheic keratosis: 0.000
squamous cell carcinoma: 0.000
melanoma: 0.000
